In [5]:
"""
Code borrowed from: 
"https://github.com/MITESHPUTHRANNEU/Speech-Emotion-Analyzer"
"https://github.com/MITESHPUTHRANNEU/Speech-Emotion-Analyzer/blob/master/saved_models/Emotion_Voice_Detection_Model.h5"
"https://github.com/MITESHPUTHRANNEU/Speech-Emotion-Analyzer/blob/master/AudioRecorder.ipynb"
"""

import pyaudio
import wave
import warnings

import ctypes
MB_DEFAULT = 0
MB_OKCANCEL = 0x00000001
MessageBox = ctypes.windll.user32.MessageBoxW
#puts out message box, once user hits ok the program begins to take in audio
MessageBox(None, 'Start recording.....              ', 'Emotion Detection', MB_OKCANCEL)  
   
#print("live recording")

continue_inference = True
while continue_inference:    
    #setting formatting for recording 4 second stream
    CHUNK = 1024 
    FORMAT = pyaudio.paInt16 
    CHANNELS = 2 
    RATE = 44100 
    RECORD_SECONDS = 4
    WAVE_OUTPUT_FILENAME = "output10.wav"

    p = pyaudio.PyAudio()

    #begins to stream
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK) #buffer


    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()
    #stream over

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    import librosa
    from librosa.display import waveplot
    import matplotlib.pyplot as plt

    #loads recording into the output10 file
    data, sampling_rate = librosa.load('output10.wav')


    import os
    import pandas as pd
    import librosa
    import glob 

    import numpy as np

    #loads the recording file, saves sample rate, and extracts elevant audio features (MFCCs) into numpy array
    X, sample_rate = librosa.load('output10.wav', res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
    featurelive = mfccs
    livedf2 = featurelive

    #data from numpy array --> pandas dataframe 
    livedf2= pd.DataFrame(data=livedf2)

    #alters dataframe formatting so rows become columns and colums become rows
    livedf2 = livedf2.stack().to_frame().T

    #adds in one more dimension to data
    twodim= np.expand_dims(livedf2, axis=2)

    import tensorflow as tf
    
    #loads weights from pretrained model
    loaded_model = tf.keras.models.load_model('C:/datasets/day 22/Emotion_Voice_Detection_Model (2).h5')

    #model makes predictions on MFCCs
    livepreds = loaded_model.predict(twodim, 
                             batch_size=32, 
                             verbose=1)
    
    #saves label predicted by model
    livepreds1=livepreds.argmax(axis=1)
    
    #ensures label formatted as integer
    liveabc = livepreds1.astype(int).flatten()

    from sklearn.preprocessing import LabelEncoder

    lb = LabelEncoder()
    #defines encoding emotion labels and has each label correspond to its position on the lists -- e.g. female_angry becomes 0 in int_labels array
    str_labels = np.array(["female_angry", "female_calm", "female_fearful", "female_happy", "female_sad", "male_angry", "male_calm", "male_fearful", "male_happy", "male_sad"])
    int_labels = lb.fit_transform(str_labels)

    #integer_label --> string label
    livepredictions = (lb.inverse_transform((liveabc)))

    #takes string label inside of the array item livepredictions
    label = livepredictions[0]
        
    #print(label.split("_",1)[1]) -- simply outputs emotion without reference to identified gender
    
    #if user hits cancel the model stops running and taking in audio
    should_cancel = MessageBox(None, label.split("_",1)[1], 'Emotion Detection', MB_OKCANCEL) 
    if should_cancel == 2:
        continue_inference = False
    
    

1/1 [==============================] - 0s 69ms/step
